In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import csv
import re
import time

In [2]:
df = pd.read_csv(r'C:\workspaces\project01\Project_Visual\restaurant_data\TotalData_J.csv')
# df

In [3]:
nm_null = df[df['BPLCNM'].isnull()].index

df.iloc[nm_null, 1] = ('엔에이')
df.iloc[nm_null]

,Unnamed: 0,BPLCNM,TRDSTATENM,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,X,Y
10449,22303,엔에이,영업/정상,100848.0,서울특별시 중구 을지로4가 35번지 2층,"서울특별시 중구 창경궁로5길 27, 2층 (을지로4가)",199614.974362,451625.623159


In [4]:
df['SITEWHLADDR'] = np.where(pd.notnull(df['SITEWHLADDR']) == True, df['SITEWHLADDR'], df['RDNWHLADDR'])

df.isnull().sum()

Unnamed: 0        0
BPLCNM            0
TRDSTATENM        0
SITEPOSTNO       80
SITEWHLADDR       0
RDNWHLADDR     1660
X              1880
Y              1880
dtype: int64

In [5]:
df1 = df[['BPLCNM','SITEWHLADDR', 'X', 'Y']]

df1

,BPLCNM,SITEWHLADDR,X,Y
0,북악파크부페,서울특별시 종로구 평창동 110-1번지,197993.205677,456584.627065
1,시민식당 2호점,서울특별시 종로구 인사동 98번지,198681.571899,452050.274345
2,일번우동,서울특별시 종로구 종로6가 129-1번지,200533.453584,452156.900283
3,서촌소국밥,서울특별시 종로구 통인동 27-2,197250.385438,453135.451266
4,빽다방 동대문역점,서울특별시 종로구 창신동 696-1,200811.925354,452162.569488
...,...,...,...,...
124905,20bucks,서울특별시 동대문구 이문동 316-2 지하1층,205294.170617,454765.261112
124906,DNJ,서울특별시 노원구 하계동 256-11 보스턴 산부인과,206490.594308,459688.060943
124907,희래등반점,서울특별시 종로구 명륜3가 1-14,199506.183606,454029.397330
124908,별달맛집,서울특별시 강남구 신사동 609-1 지하1층,202534.550000,447079.205000


In [6]:
BPL = df1['BPLCNM'].values[:]
BP = list(BPL)

In [7]:
BP

['북악파크부페',
 '시민식당 2호점',
 '일번우동',
 '서촌소국밥',
 '빽다방 동대문역점',
 '팔도식당',
 '군산집',
 '올리브떡볶이(동묘점)',
 '소문',
 '주식회사 가배도 삼청',
 '강원맛코다리',
 '이화돈까스',
 '학아재',
 '기생',
 '마이기어 종로점',
 '상구네백반',
 '스쿨푸드 딜리버리 종로',
 '서울대병원 델라코트',
 '서울대병원 가화원',
 '홈푸드 스토리',
 '한옥집 김치찜',
 '맷돌로만 숭인점',
 '로열네팔레스토랑&바르Ⅱ',
 '장수통닭',
 '호센',
 '뉴오리진 광화문점',
 '나인티나인',
 '오쵸',
 '뜰채',
 '아르크',
 '행운돈까스지기',
 '미요시',
 '아담집',
 '스튜디오 이여로',
 '계동만담',
 '백석동주',
 '퀴진 라끌레(Cuisine La cle)',
 '도쿄술집 종로관철점',
 '두레유',
 '연정',
 '어심',
 '애드',
 '국시방',
 '우미옥',
 '시인',
 '가르텐 비어',
 '명륜 쭈꾸미',
 '파닭',
 '곰돌이',
 '홍콩반점 0410',
 '미성',
 '효제',
 '뚜레쥬르',
 '장수촌 풍천장어',
 '만미옥',
 '블루몬테',
 '지하철호프',
 '정통삼계탕',
 '한성집',
 '솜씨맛',
 '마님은왜돌쇠에게만술을주시나',
 '한잔할청춘아',
 '잃어버린도시마야',
 '메밀꽃필무렵',
 '비비큐 대학로 명륜점',
 '가정식당',
 '써브웨이 (광화문점)',
 '유성집',
 '수왕',
 '장수촌',
 '먹거리집',
 '참나무동산(OAK HILL)',
 '문가네',
 '호',
 '홍아네',
 '청원식당',
 '반저',
 '경수사',
 '비비큐 치킨(인사점)',
 '서촌 기와집',
 '궁전',
 '부산집 막국수',
 '무지개',
 '등촌샤브 칼국수 안국역점',
 '원조쌈밥집',
 '아람',
 '부산 아구찜',
 '천석식당',
 '경성',
 '딩딤1968 종각센트로폴리스점',
 '경희궁',
 '익선 통하다',
 '옛날집',
 '송죽헌',
 '왕벌집',


In [8]:
BPP = [i.replace(' ', '') for i in BP]
BPP

['북악파크부페',
 '시민식당2호점',
 '일번우동',
 '서촌소국밥',
 '빽다방동대문역점',
 '팔도식당',
 '군산집',
 '올리브떡볶이(동묘점)',
 '소문',
 '주식회사가배도삼청',
 '강원맛코다리',
 '이화돈까스',
 '학아재',
 '기생',
 '마이기어종로점',
 '상구네백반',
 '스쿨푸드딜리버리종로',
 '서울대병원델라코트',
 '서울대병원가화원',
 '홈푸드스토리',
 '한옥집김치찜',
 '맷돌로만숭인점',
 '로열네팔레스토랑&바르Ⅱ',
 '장수통닭',
 '호센',
 '뉴오리진광화문점',
 '나인티나인',
 '오쵸',
 '뜰채',
 '아르크',
 '행운돈까스지기',
 '미요시',
 '아담집',
 '스튜디오이여로',
 '계동만담',
 '백석동주',
 '퀴진라끌레(CuisineLacle)',
 '도쿄술집종로관철점',
 '두레유',
 '연정',
 '어심',
 '애드',
 '국시방',
 '우미옥',
 '시인',
 '가르텐비어',
 '명륜쭈꾸미',
 '파닭',
 '곰돌이',
 '홍콩반점0410',
 '미성',
 '효제',
 '뚜레쥬르',
 '장수촌풍천장어',
 '만미옥',
 '블루몬테',
 '지하철호프',
 '정통삼계탕',
 '한성집',
 '솜씨맛',
 '마님은왜돌쇠에게만술을주시나',
 '한잔할청춘아',
 '잃어버린도시마야',
 '메밀꽃필무렵',
 '비비큐대학로명륜점',
 '가정식당',
 '써브웨이(광화문점)',
 '유성집',
 '수왕',
 '장수촌',
 '먹거리집',
 '참나무동산(OAKHILL)',
 '문가네',
 '호',
 '홍아네',
 '청원식당',
 '반저',
 '경수사',
 '비비큐치킨(인사점)',
 '서촌기와집',
 '궁전',
 '부산집막국수',
 '무지개',
 '등촌샤브칼국수안국역점',
 '원조쌈밥집',
 '아람',
 '부산아구찜',
 '천석식당',
 '경성',
 '딩딤1968종각센트로폴리스점',
 '경희궁',
 '익선통하다',
 '옛날집',
 '송죽헌',
 '왕벌집',
 '웃음의식당',
 '꽃피는산골',
 '한일',
 '삼형제김밥

In [38]:
df2 =pd.DataFrame(BPP)
df2.columns = ['BPLCNM']
df2

,BPLCNM
0,북악파크부페
1,시민식당2호점
2,일번우동
3,서촌소국밥
4,빽다방동대문역점
...,...
124905,20bucks
124906,DNJ
124907,희래등반점
124908,별달맛집


In [9]:
SITE = df1['SITEWHLADDR'].values[:]
SI = list(SITE)

In [10]:
SI

['서울특별시 종로구 평창동 110-1번지 ',
 '서울특별시 종로구 인사동 98번지 ',
 '서울특별시 종로구 종로6가 129-1번지 ',
 '서울특별시 종로구 통인동 27-2 ',
 '서울특별시 종로구 창신동 696-1 ',
 '서울특별시 종로구 묘동 35-1번지 ',
 '서울특별시 종로구 교북동 90-1번지 ',
 '서울특별시 종로구 숭인동 65-18번지 지상1층 ',
 '서울특별시 종로구 공평동 5-1번지 ',
 '서울특별시 종로구 소격동 142-2 ',
 '서울특별시 종로구 숭인동 212-1번지 ',
 '서울특별시 종로구 안국동 159 사비나미술관 ',
 '서울특별시 종로구 통인동 135-2번지 ',
 '서울특별시 종로구 충신동 1-134 ',
 '서울특별시 종로구 종로5가 225-18번지 ',
 '서울특별시 종로구 무악동 82번지 무악현대아파트 ',
 '서울특별시 종로구 효제동 15-1 대건빌딩 ',
 '서울특별시 종로구 연건동 28-21 서울대학교 의과대학 ',
 '서울특별시 종로구 연건동 28-21 서울대학교 의과대학 ',
 '서울특별시 종로구 숭인동 336 1층 ',
 '서울특별시 종로구 공평동 17번지 ',
 '서울특별시 종로구 숭인동 200-33번지 엘리시앙오피스텔 1층 ',
 '서울특별시 종로구 창신동 151-1 지하1층 ',
 '서울특별시 종로구 수송동 80-6번지 석탄회관빌딩 ',
 '서울특별시 종로구 사직동 9번지 광화문 풍림스페이스본 ',
 '서울특별시 종로구 신문로1가 135-2번지 ',
 '서울특별시 종로구 혜화동 185번지 중원빌딩 ',
 '서울특별시 종로구 도렴동 150-1번지 ',
 '서울특별시 종로구 숭인동 201-7번지 ',
 '서울특별시 종로구 사직동 262-4번지 ',
 '서울특별시 종로구 돈의동 137 피카디리플러스 ',
 '서울특별시 종로구 사직동 9번지 광화문 풍림스페이스본 ',
 '서울특별시 종로구 익선동 154-17 ',
 '서울특별시 종로구 구기동 54-13번지 ',
 '서울특별시 종로구 계동 140-41

In [11]:
df1['SITEWHLADDR'] = df1['SITEWHLADDR'].astype('str')
df1['SITEWHLADDR'].dtypes

C:\tmp/ipykernel_25572/3052405309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['SITEWHLADDR'] = df1['SITEWHLADDR'].astype('str')


dtype('O')

In [12]:
df_split = df1['SITEWHLADDR'].str.split(' ', expand=True)
df_split.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,서울특별시,종로구,평창동,110-1번지,,None,None,None,None,None,None,None,None,None,None,None
1,서울특별시,종로구,인사동,98번지,,None,None,None,None,None,None,None,None,None,None,None
2,서울특별시,종로구,종로6가,129-1번지,,None,None,None,None,None,None,None,None,None,None,None
3,서울특별시,종로구,통인동,27-2,,None,None,None,None,None,None,None,None,None,None,None
4,서울특별시,종로구,창신동,696-1,,None,None,None,None,None,None,None,None,None,None,None


In [13]:
dfs = df_split.iloc[:, 0:3]

In [14]:
dfs.columns = ['시도', '구', '동']
dfs.head()

,시도,구,동
0,서울특별시,종로구,평창동
1,서울특별시,종로구,인사동
2,서울특별시,종로구,종로6가
3,서울특별시,종로구,통인동
4,서울특별시,종로구,창신동


In [15]:
dfs

,시도,구,동
0,서울특별시,종로구,평창동
1,서울특별시,종로구,인사동
2,서울특별시,종로구,종로6가
3,서울특별시,종로구,통인동
4,서울특별시,종로구,창신동
...,...,...,...
124905,서울특별시,동대문구,이문동
124906,서울특별시,노원구,하계동
124907,서울특별시,종로구,명륜3가
124908,서울특별시,강남구,신사동


In [16]:
SITEWHLADDR = list( dfs["시도"] + " " + dfs["구"] + " " + dfs["동"] )
SITEWHLADDR

['서울특별시 종로구 평창동',
 '서울특별시 종로구 인사동',
 '서울특별시 종로구 종로6가',
 '서울특별시 종로구 통인동',
 '서울특별시 종로구 창신동',
 '서울특별시 종로구 묘동',
 '서울특별시 종로구 교북동',
 '서울특별시 종로구 숭인동',
 '서울특별시 종로구 공평동',
 '서울특별시 종로구 소격동',
 '서울특별시 종로구 숭인동',
 '서울특별시 종로구 안국동',
 '서울특별시 종로구 통인동',
 '서울특별시 종로구 충신동',
 '서울특별시 종로구 종로5가',
 '서울특별시 종로구 무악동',
 '서울특별시 종로구 효제동',
 '서울특별시 종로구 연건동',
 '서울특별시 종로구 연건동',
 '서울특별시 종로구 숭인동',
 '서울특별시 종로구 공평동',
 '서울특별시 종로구 숭인동',
 '서울특별시 종로구 창신동',
 '서울특별시 종로구 수송동',
 '서울특별시 종로구 사직동',
 '서울특별시 종로구 신문로1가',
 '서울특별시 종로구 혜화동',
 '서울특별시 종로구 도렴동',
 '서울특별시 종로구 숭인동',
 '서울특별시 종로구 사직동',
 '서울특별시 종로구 돈의동',
 '서울특별시 종로구 사직동',
 '서울특별시 종로구 익선동',
 '서울특별시 종로구 구기동',
 '서울특별시 종로구 계동',
 '서울특별시 종로구 평창동',
 '서울특별시 종로구 통인동',
 '서울특별시 종로구 관철동',
 '서울특별시 종로구 평창동',
 '서울특별시 종로구 내수동',
 '서울특별시 종로구 낙원동',
 '서울특별시 종로구 세종로',
 '서울특별시 종로구 당주동',
 '서울특별시 종로구 충신동',
 '서울특별시 종로구 신문로2가',
 '서울특별시 종로구 종로3가',
 '서울특별시 종로구 명륜3가',
 '서울특별시 종로구 명륜4가',
 '서울특별시 종로구 명륜3가',
 '서울특별시 종로구 동숭동',
 '서울특별시 종로구 운니동',
 '서울특별시 종로구 연건동',
 '서울특별시 종로구 명륜4가',
 '서울특별시 종로구 관철동',
 '서울특별시 종로구 봉익동',
 

In [17]:
df1["SITEWHLADDR"] = SITEWHLADDR

C:\tmp/ipykernel_25572/4035550849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["SITEWHLADDR"] = SITEWHLADDR


In [18]:
df1

,BPLCNM,SITEWHLADDR,X,Y
0,북악파크부페,서울특별시 종로구 평창동,197993.205677,456584.627065
1,시민식당 2호점,서울특별시 종로구 인사동,198681.571899,452050.274345
2,일번우동,서울특별시 종로구 종로6가,200533.453584,452156.900283
3,서촌소국밥,서울특별시 종로구 통인동,197250.385438,453135.451266
4,빽다방 동대문역점,서울특별시 종로구 창신동,200811.925354,452162.569488
...,...,...,...,...
124905,20bucks,서울특별시 동대문구 이문동,205294.170617,454765.261112
124906,DNJ,서울특별시 노원구 하계동,206490.594308,459688.060943
124907,희래등반점,서울특별시 종로구 명륜3가,199506.183606,454029.397330
124908,별달맛집,서울특별시 강남구 신사동,202534.550000,447079.205000


In [19]:
df1.isnull().sum()

BPLCNM            0
SITEWHLADDR       0
X              1880
Y              1880
dtype: int64

* 네이버 api를 통한 좌표 마킹 + 상세 정보 표현이 필요하다

* 가게명과 주소를 통한 크롤링으로 가게들의 메뉴와 가격정보를 가져와야한다

* 이 가격정보를 통해 지도에 마킹한곳에 대한 상제정보 도출에 활용

In [39]:
BS = list( df1["SITEWHLADDR"] + " " + df2["BPLCNM"] )
BS

['서울특별시 종로구 평창동 북악파크부페',
 '서울특별시 종로구 인사동 시민식당2호점',
 '서울특별시 종로구 종로6가 일번우동',
 '서울특별시 종로구 통인동 서촌소국밥',
 '서울특별시 종로구 창신동 빽다방동대문역점',
 '서울특별시 종로구 묘동 팔도식당',
 '서울특별시 종로구 교북동 군산집',
 '서울특별시 종로구 숭인동 올리브떡볶이(동묘점)',
 '서울특별시 종로구 공평동 소문',
 '서울특별시 종로구 소격동 주식회사가배도삼청',
 '서울특별시 종로구 숭인동 강원맛코다리',
 '서울특별시 종로구 안국동 이화돈까스',
 '서울특별시 종로구 통인동 학아재',
 '서울특별시 종로구 충신동 기생',
 '서울특별시 종로구 종로5가 마이기어종로점',
 '서울특별시 종로구 무악동 상구네백반',
 '서울특별시 종로구 효제동 스쿨푸드딜리버리종로',
 '서울특별시 종로구 연건동 서울대병원델라코트',
 '서울특별시 종로구 연건동 서울대병원가화원',
 '서울특별시 종로구 숭인동 홈푸드스토리',
 '서울특별시 종로구 공평동 한옥집김치찜',
 '서울특별시 종로구 숭인동 맷돌로만숭인점',
 '서울특별시 종로구 창신동 로열네팔레스토랑&바르Ⅱ',
 '서울특별시 종로구 수송동 장수통닭',
 '서울특별시 종로구 사직동 호센',
 '서울특별시 종로구 신문로1가 뉴오리진광화문점',
 '서울특별시 종로구 혜화동 나인티나인',
 '서울특별시 종로구 도렴동 오쵸',
 '서울특별시 종로구 숭인동 뜰채',
 '서울특별시 종로구 사직동 아르크',
 '서울특별시 종로구 돈의동 행운돈까스지기',
 '서울특별시 종로구 사직동 미요시',
 '서울특별시 종로구 익선동 아담집',
 '서울특별시 종로구 구기동 스튜디오이여로',
 '서울특별시 종로구 계동 계동만담',
 '서울특별시 종로구 평창동 백석동주',
 '서울특별시 종로구 통인동 퀴진라끌레(CuisineLacle)',
 '서울특별시 종로구 관철동 도쿄술집종로관철점',
 '서울특별시 종로구 평창동 두레유',
 '서울특별시 종로구 내수동 연정',
 '서울특별

In [40]:
result_list = []

for i in BS:
    temp = i.replace("메가엠지씨커피", "메가커피")
    result_list.append(temp)

In [41]:
BSdf = pd.DataFrame(result_list)
BSdf

,0
0,서울특별시 종로구 평창동 북악파크부페
1,서울특별시 종로구 인사동 시민식당2호점
2,서울특별시 종로구 종로6가 일번우동
3,서울특별시 종로구 통인동 서촌소국밥
4,서울특별시 종로구 창신동 빽다방동대문역점
...,...
124905,서울특별시 동대문구 이문동 20bucks
124906,서울특별시 노원구 하계동 DNJ
124907,서울특별시 종로구 명륜3가 희래등반점
124908,서울특별시 강남구 신사동 별달맛집


In [42]:
BSdf.columns = ['주소 + 가게명']
BSdf.head()

,주소 + 가게명
0,서울특별시 종로구 평창동 북악파크부페
1,서울특별시 종로구 인사동 시민식당2호점
2,서울특별시 종로구 종로6가 일번우동
3,서울특별시 종로구 통인동 서촌소국밥
4,서울특별시 종로구 창신동 빽다방동대문역점


In [43]:
inlist = list[BSdf["주소 + 가게명"]]
print(inlist)

list[0              서울특별시 종로구 평창동 북악파크부페
1             서울특별시 종로구 인사동 시민식당2호점
2               서울특별시 종로구 종로6가 일번우동
3               서울특별시 종로구 통인동 서촌소국밥
4            서울특별시 종로구 창신동 빽다방동대문역점
                    ...            
124905       서울특별시 동대문구 이문동 20bucks
124906            서울특별시 노원구 하계동 DNJ
124907         서울특별시 종로구 명륜3가 희래등반점
124908           서울특별시 강남구 신사동 별달맛집
124909    서울특별시 용산구 한강로2가 메가커피신용산역점
Name: 주소 + 가게명, Length: 124910, dtype: object]


In [44]:
BSdf["주소 + 가게명"]
print(BSdf["주소 + 가게명"])     
print(type(BSdf["주소 + 가게명"]))
print(BSdf["주소 + 가게명"][123])  

0              서울특별시 종로구 평창동 북악파크부페
1             서울특별시 종로구 인사동 시민식당2호점
2               서울특별시 종로구 종로6가 일번우동
3               서울특별시 종로구 통인동 서촌소국밥
4            서울특별시 종로구 창신동 빽다방동대문역점
                    ...            
124905       서울특별시 동대문구 이문동 20bucks
124906            서울특별시 노원구 하계동 DNJ
124907         서울특별시 종로구 명륜3가 희래등반점
124908           서울특별시 강남구 신사동 별달맛집
124909    서울특별시 용산구 한강로2가 메가커피신용산역점
Name: 주소 + 가게명, Length: 124910, dtype: object
<class 'pandas.core.series.Series'>
서울특별시 종로구 종로5가 타이거슈가종로5가점


In [45]:
print(type(list(BSdf["주소 + 가게명"])))
print(list(BSdf["주소 + 가게명"][7847]))

<class 'list'>
['서', '울', '특', '별', '시', ' ', '중', '구', ' ', '필', '동', '1', '가', ' ', '남', '산', '골', '생', '고', '기']


In [46]:
qie = BSdf.values.tolist()
qie[123]

['서울특별시 종로구 종로5가 타이거슈가종로5가점']

In [47]:
print(str(BSdf["주소 + 가게명"]))
print(type(str(BSdf["주소 + 가게명"])))
print(str(BSdf["주소 + 가게명"][48]))
print(type(str(BSdf["주소 + 가게명"][48])))

0              서울특별시 종로구 평창동 북악파크부페
1             서울특별시 종로구 인사동 시민식당2호점
2               서울특별시 종로구 종로6가 일번우동
3               서울특별시 종로구 통인동 서촌소국밥
4            서울특별시 종로구 창신동 빽다방동대문역점
                    ...            
124905       서울특별시 동대문구 이문동 20bucks
124906            서울특별시 노원구 하계동 DNJ
124907         서울특별시 종로구 명륜3가 희래등반점
124908           서울특별시 강남구 신사동 별달맛집
124909    서울특별시 용산구 한강로2가 메가커피신용산역점
Name: 주소 + 가게명, Length: 124910, dtype: object
<class 'str'>
서울특별시 종로구 명륜3가 곰돌이
<class 'str'>


In [48]:
inlist = BSdf["주소 + 가게명"][124909]
inlist

'서울특별시 용산구 한강로2가 메가커피신용산역점'

In [49]:
print(type(str(BSdf["주소 + 가게명"][0])))

<class 'str'>


In [50]:
inlist = list(BSdf["주소 + 가게명"])
inlist[0]

'서울특별시 종로구 평창동 북악파크부페'

In [32]:
driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
driver.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=")

time.sleep(3)

search_box = driver.find_element_by_css_selector("#nx_search_form > fieldset > div.greenbox")

time.sleep(3)

x_path = '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div'

search = driver.find_element_by_xpath(x_path)

search.click()

element = driver.find_element_by_xpath("/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input")

element.send_keys("자장면")

element.send_keys(Keys.ENTER)

time.sleep(3)

# 막아둠..
store = driver.find_element_by_class_name("_3hn9q")

store.click()

C:\tmp/ipykernel_25572/2941420562.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
C:\tmp/ipykernel_25572/2941420562.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search_box = driver.find_element_by_css_selector("#nx_search_form > fieldset > div.greenbox")
C:\tmp/ipykernel_25572/2941420562.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search = driver.find_element_by_xpath(x_path)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div"}
  (Session info: chrome=103.0.5060.66)
Stacktrace:
Backtrace:
	Ordinal0 [0x00856463+2188387]
	Ordinal0 [0x007EE461+1762401]
	Ordinal0 [0x00703D78+802168]
	Ordinal0 [0x00731880+989312]
	Ordinal0 [0x00731B1B+989979]
	Ordinal0 [0x0075E912+1173778]
	Ordinal0 [0x0074C824+1099812]
	Ordinal0 [0x0075CC22+1166370]
	Ordinal0 [0x0074C5F6+1099254]
	Ordinal0 [0x00726BE0+945120]
	Ordinal0 [0x00727AD6+948950]
	GetHandleVerifier [0x00AF71F2+2712546]
	GetHandleVerifier [0x00AE886D+2652765]
	GetHandleVerifier [0x008E002A+520730]
	GetHandleVerifier [0x008DEE06+516086]
	Ordinal0 [0x007F468B+1787531]
	Ordinal0 [0x007F8E88+1805960]
	Ordinal0 [0x007F8F75+1806197]
	Ordinal0 [0x00801DF1+1842673]
	BaseThreadInitThunk [0x75C5FA29+25]
	RtlGetAppContainerNamedObjectPath [0x778E7A9E+286]
	RtlGetAppContainerNamedObjectPath 

In [101]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

class SearchBot:
    
    def __init__(self):
        
        self.driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
        self.driver.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=")
    
    
    def Search1(self, Key, ide, name):
        
        for Keyword in tqdm(Key):
    
            element = self.driver.find_element_by_id(ide)
            element.send_keys(str(Keyword))

            time.sleep(3)

            element_1 = self.driver.find_element_by_class_name(name)
            element_1.click()
        
    def TryBot(self, nm, path):
        try:
            nm = self.driver.find_element_by_xpath(str(path)).text
        except:
            nm = " "
            
        time.sleep(2)
        
    def upTryBot(self, sl, mn1, mn2, fb, ph1, ph2):
        dt = sl
        
        for i in sl:
            
            try:
                mn1 = self.driver.fb(ph1).text
                mn2 = self.driver.fb(ph2).text
        
            except:
                mn1 = None
                mn2 = None

In [79]:
final_result = []


i = 0
while i <= 10:
    
    driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
    
    driver.get("https://naver.com")

    time.sleep(3)
    
    qie = BSdf.values.tolist()

    Keyst = qie[i]
    
    for Keyword in tqdm(Keyst):
    
        elem = driver.find_element_by_id('query')
        elem.send_keys(str(Keyword))

        time.sleep(3)

        eleme = driver.find_element_by_class_name('btn_submit')
        eleme.click()
        
        try:
            name = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[2]/div[1]/div[1]/a/span[1]').text
        except:
            name = " "
            
        time.sleep(2)
            
        try:
            stype = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[2]/div[1]/div[1]/a/span[2]').text
        except:
            stype = " "
            
        time.sleep(2)
            
        try:
            starn = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[2]/div[1]/div[2]/span[1]/em').text
        except:
            starn = " "
        
        time.sleep(2)
        
        try:
            calln = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[3]/div/ul/li[1]/div/span[1]').text
        except:
            calln = " "
            
        time.sleep(2)
        
        try:
            ad = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[3]/div/ul/li[2]/div/a/span[1]').text
        except:
            ad = " "
            
        time.sleep(2)
        
        try:
            stm = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[3]/div/ul/li[3]/div/a/div/div/div/span/time').text
        except:
            stm = " "        
       
                    
        store_info = {
            "storename":name, 
            "storetype":stype, 
            "starnum":starn, 
            "callnum":calln, 
            "address":ad, 
            "info_time":stm
        }
        
        final_result.append(store_info)
    
    i = i + 1
    
    time.sleep(3)
    
    driver.close()

C:\tmp/ipykernel_25572/4246414464.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\tmp/ipykernel_25572/4246414464.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_id('query')
C:\tmp/ipykernel_25572/4246414464.py:24: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  eleme = driver.find_element_by_class_name('btn_submit')
C:\tmp/ipykernel_25572/4246414464.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[2]/div[1]/d

NameError: name 'string_list' is not defined

In [ ]:
file = open('stores.csv', mode='w', newline='')
writer = csv.writer(file)
writer.writerow(["storename", "storetype", "starnum", "callnum", "address", "info_time", "image"])





for result in final_result:
    row = []
    row.append(result['storename'])
    row.append(result['storetype'])
    row.append(result['starnum'])
    row.append(result['callnum'])
    row.append(result['address'])
    row.append(result['info_time'])
    writer.writerow(row)

In [76]:
print(dictionary)

NameError: name 'dictionary' is not defined

In [77]:
print(name, stype, starn, calln, ad, stm)

NameError: name 'name' is not defined

In [78]:
print(final_result)

NameError: name 'final_result' is not defined

In [128]:
menu_result = []
s_menu1 = str()
s_pr1 = str()
s_menu2 = str()
s_pr2 = str()
mp_menu1 = str()
mp_menu2 = str()

i = 55
while i <= 56:
    
    driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
    
    driver.get("https://naver.com")

    time.sleep(3)
    
    qie = BSdf.values.tolist()

    Keyst = qie[i]
    
    for Keyword in tqdm(Keyst):
        
        elem = driver.find_element_by_id('query')
        elem.send_keys("서울특별시 종로구 명륜3가 희래등반점")

        time.sleep(2)

        eleme = driver.find_element_by_class_name('btn_submit')
        eleme.click()
        
        time.sleep(5)
        
        try:
            s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/div/span/a').text
            s_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/div/span/a').text

            s_pr1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/em').text
            s_pr2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/em').text
        
        except:
            pass
        
        time.sleep(2)
        
        if s_menu1 == None:
            
            try:
                s_menu1 = driver.find_element_by_xpath('//*[@class="_1WK4L _2EEOB"]/li[1]/div/div[1]/span/a').text
                s_menu2 = driver.find_element_by_xpath('//*[@class="_1WK4L _2EEOB"]/li[2]/div/div[1]/span/a').text

                s_pr1 = driver.find_element_by_xpath('//*[@class="_1WK4L _2EEOB"]/li[1]/div/div[2]').text
                s_pr2 = driver.find_element_by_xpath('//*[@class="_1WK4L _2EEOB"]/li[2]/div/div[2]').text
                
            except:
                s_menu1 = " "
                s_menu2 = " "
                
                s_pr1 = " "
                s_pr2 = " "                
                pass
            
        else:
            pass
        
        time.sleep(2)
        
        if s_menu1 == None:
            
            try:
                mp_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[3]/div[2]/div[4]/div/span[1]/text()')
                mp_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[3]/div[2]/div[4]/div/span[2]').text
                
            except:
                mp_menu1 = " " 
                mp_menu2 = " "
                pass

        else:
            pass
        
        time.sleep(2)
        
        menu_info = {
            "menuname1":s_menu1, 
            "menupr1":s_pr1,
            "menuname2":s_menu2,
            "menupr2":s_pr2,
            "mp1":mp_menu1,
            "mp2":mp_menu2
        }
        
        time.sleep(2)
        
        menu_result.append(menu_info)
        
    i = i + 1
        
    time.sleep(3)
        
    driver.close()

C:\tmp/ipykernel_25572/1576875860.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\tmp/ipykernel_25572/1576875860.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_id('query')
C:\tmp/ipykernel_25572/1576875860.py:29: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  eleme = driver.find_element_by_class_name('btn_submit')
C:\tmp/ipykernel_25572/1576875860.py:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[

KeyboardInterrupt: 

In [129]:
print(menu_info)

{'menuname1': '', 'menupr1': '', 'menuname2': '', 'menupr2': '', 'mp1': '', 'mp2': ''}


In [130]:
print(menu_result)

[{'menuname1': '', 'menupr1': '', 'menuname2': '', 'menupr2': '', 'mp1': '', 'mp2': ''}]


In [142]:
menu_result = []
s_menu1 = str()
s_pr1 = str()
s_menu2 = str()
s_pr2 = str()
mp_menu1 = str()
mp_menu2 = str()

i = 0
while i <= 99:
    
    driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
    
    driver.get("https://naver.com")

    time.sleep(3)
    
    qie = BSdf.values.tolist()

    Keyst = qie[i]
    
    for Keyword in tqdm(Keyst):
        
        elem = driver.find_element_by_id('query')
        elem.send_keys(str(Keyword))

        time.sleep(1)

        eleme = driver.find_element_by_class_name('btn_submit')
        eleme.click()
        
        time.sleep(1)
        
        try:
            s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/div/span/a').text
            s_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/div/span/a').text

            s_pr1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/em').text
            s_pr2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/em').text
        
        except:
            try:
                s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[1]/div/div[1]/span/a').text
                s_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[2]/div/div[1]/span/a').text

                s_pr1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[1]/div/div[2]').text
                s_pr2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[2]/div/div[2]').text
            
            except:
                try:
                    mp_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[3]/div[2]/div[4]/div/span[1]').text
                    mp_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[3]/div[2]/div[4]/div/span[2]').text
                    
                except:                    
                    s_menu1 = " "
                    s_pr1 = " "
                    s_menu2 = " "
                    s_pr2 = " "
                    mp_menu1 = " "
                    mp_menu2 = " "   
                
        time.sleep(1)
        
        menu_info = {
            "menuname1":s_menu1, 
            "menupr1":s_pr1,
            "menuname2":s_menu2,
            "menupr2":s_pr2,
            "mp1":mp_menu1,
            "mp2":mp_menu2
        }
        
        time.sleep(1)
        
        menu_result.append(menu_info)
        
    i = i + 1
        
    time.sleep(1)
        
    driver.close()
        

C:\tmp/ipykernel_25572/1191824204.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\tmp/ipykernel_25572/1191824204.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_id('query')
C:\tmp/ipykernel_25572/1191824204.py:29: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  eleme = driver.find_element_by_class_name('btn_submit')
C:\tmp/ipykernel_25572/1191824204.py:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.05s/it]


In [143]:
print(menu_info)

{'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}


In [144]:
print(menu_result)

[{'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': '아메리카노HOT', 'menupr1': '1,500원', 'menuname2': '아메리카노ICE', 'menupr2': '2,000원', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': ' ', 'menupr1': ' ', 'menuname2': ' ', 'menupr2': ' ', 'mp1': ' ', 'mp2': ' '}, {'menuname1': '말차 라떼', 'menupr1': '6,500원', 'menuname2': '티라미수(일반/말차/얼그레이)', 'menupr2': '7,000원', 'mp1': ' ', 'mp2': ' '}, {'menu

In [95]:
import SearchBot as sb

qie = BSdf.values.tolist()
Key = qie[i]

ide = 'query'
name = 'btn_submit'

sl = range(1, 5)

mns1 = [s_menu1, s_menu3]
mns2 = [s_menu2, s_menu4]

mn1 = mns1[0]
mn2 = mns2[0]

fb = driver.find_element_by_xpath

phc1 = [
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/div/span/a',
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/div/span/a',
    
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[1]/div/div[1]/span/a',
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[2]/div/div[1]/span/a'
]
phc2 = [
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[1]/div/em',
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li[2]/div/em',
    
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[1]/div/div[2]',
    '/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li[2]/div/div[2]'
]

ph1 = phc1[0]
ph2 = phc2[2]

Bot = sb.SearchBot()


menu_result = []




i = 10
while i <= 10:
    
    Bot.

    time.sleep(3)
    
    qie = BSdf.values.tolist()

    Keyst = "한옥집김치찜"
    
    for j in range(1, 5):
    
        elem = driver.find_element_by_id('query')
        elem.send_keys(str(Keyst))

        time.sleep(2)

        eleme = driver.find_element_by_class_name('btn_submit')
        eleme.click()
        
        time.sleep(2)
        
        try:
            s_menu1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/div/span/a').text
            s_pr1 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/em').text
            
        except:
            s_menu1 =' '
            s_pr1 =' '
        
        time.sleep(2)
        
        try:
            s_menu2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/div/span/a').text
            s_pr2 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/em').text
            
        except:
            s_menu2 =' '
            s_pr2 =' '
        
        time.sleep(2)
        
        try:
            s_menu3 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/div/span/a').text
            s_pr3 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/em').text
            
        except:
            s_menu3 =' '
            s_pr3 =' '
        
        time.sleep(2)
        
        try:
            s_menu4 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/div/span/a').text
            s_pr4 = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/em').text
            
        except:
            s_menu4 =' '
            s_pr4 =' '
        
        time.sleep(2)
        
        
            s_menu = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li['+str(j)+']/div/div[1]/span/a').text
            s_pr = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[2]/div/ul/li[4]/div/ul/li['+str(j)+']/div/div[2]').text
            mp_menu = ' '
        
        time.sleep(2)
            
        if not mp_menu == ' ':
            
            mp_menu = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/section/div/div[3]/div[2]/div[4]/div/span['+str(j)+']').text   
            s_menu = ' '
            s_pr = ' '
            
        else:
            pass
        
        time.sleep(1)
                
        menu_info = {
            "menuname":s_menu, 
            "menupr":s_pr,
            "mp":mp_menu
        }
        
        menu_result.append(menu_info)
            
    i = i + 1
    
    time.sleep(1)
    
    driver.close()

C:\tmp/ipykernel_25572/502205434.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\jsnz3\python\chromedriver.exe")
C:\tmp/ipykernel_25572/502205434.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_id('query')
C:\tmp/ipykernel_25572/502205434.py:21: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  eleme = driver.find_element_by_class_name('btn_submit')
C:\tmp/ipykernel_25572/502205434.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  s_menu = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div[2]/div[3]/div/section/div/div[4]/div[1]/div/div/ul/li['+str(j)+']/div/div/span/a').text
C:\tmp/ipykernel_25572/502205434.py:27: DeprecationWarning: find_element_b

In [92]:
print(menu_info)

{'menuname': ' ', 'menupr': ' ', 'mp': ' '}


In [93]:
print(menu_result)

[{'menuname': ' ', 'menupr': ' ', 'mp': ' '}]
